## Modelling

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv("cleaned_data.csv")
df.head()

,Anime_id,Title,Genre,Synopsis,Type,Producer,Studio,Rating,ScoredBy,Popularity,Members,Episodes,Source,Aired
0,1,Cowboy Bebop,"['Action', 'Adventure', 'Comedy', 'Drama', 'Sc...","In the year 2071, humanity has colonized sever...",TV,['Bandai Visual'],['Sunrise'],8.81,363889.0,39.0,704490.0,26.0,Original,"Apr 3, 1998 to Apr 24, 1999"
1,5,Cowboy Bebop Tengoku no Tobira,"['Action', 'Space', 'Drama', 'Mystery', 'Sci-Fi']","Another day, another bounty—such is the life o...",Movie,"['Sunrise', 'Bandai Visual']",['Bones'],8.41,111187.0,475.0,179899.0,1.0,Original,"Sep 1, 2001"
2,6,Trigun,"['Action', 'Sci-Fi', 'Adventure', 'Comedy', 'D...","Vash the Stampede is the man with a $$60,000,0...",TV,['Victor Entertainment'],['Madhouse'],8.31,197451.0,158.0,372709.0,26.0,Manga,"Apr 1, 1998 to Sep 30, 1998"
3,7,Witch Hunter Robin,"['Action', 'Magic', 'Police', 'Supernatural', ...",Witches are individuals with special powers li...,TV,['Bandai Visual'],['Sunrise'],7.34,31875.0,1278.0,74889.0,26.0,Original,"Jul 2, 2002 to Dec 24, 2002"
4,8,Bouken Ou Beet,"['Adventure', 'Fantasy', 'Shounen', 'Supernatu...",It is the dark century and the people are suff...,TV,NaN,['Toei Animation'],7.04,4757.0,3968.0,11247.0,52.0,Manga,"Sep 30, 2004 to Sep 29, 2005"


In [5]:
!! pip install scikit-learn

['Requirement already satisfied: scikit-learn in c:\\users\\hp\\documents\\anaconda\\lib\\site-packages (1.1.3)',
 'Requirement already satisfied: numpy>=1.17.3 in c:\\users\\hp\\documents\\anaconda\\lib\\site-packages (from scikit-learn) (1.21.5)',
 'Requirement already satisfied: scipy>=1.3.2 in c:\\users\\hp\\documents\\anaconda\\lib\\site-packages (from scikit-learn) (1.7.3)',
 'Requirement already satisfied: threadpoolctl>=2.0.0 in c:\\users\\hp\\documents\\anaconda\\lib\\site-packages (from scikit-learn) (2.2.0)',
 'Requirement already satisfied: joblib>=1.0.0 in c:\\users\\hp\\documents\\anaconda\\lib\\site-packages (from scikit-learn) (1.1.0)']

In [6]:
df['Synopsis'].isnull().sum()

0

In [7]:
df['Synopsis'] = df['Synopsis'].fillna('')

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')

tfidf_matrix = tfidf.fit_transform(df['Synopsis'])

tfidf_matrix.shape

(12979, 40034)

In [9]:
from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim

array([[1.        , 0.23529586, 0.02127464, ..., 0.02133734, 0.00457067,
        0.02050118],
       [0.23529586, 1.        , 0.04320112, ..., 0.01628467, 0.01529336,
        0.        ],
       [0.02127464, 0.04320112, 1.        , ..., 0.00264197, 0.        ,
        0.        ],
       ...,
       [0.02133734, 0.01628467, 0.00264197, ..., 1.        , 0.        ,
        0.        ],
       [0.00457067, 0.01529336, 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.02050118, 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [10]:
indices = pd.Series(df.index, index=df['Title']).drop_duplicates()
indices

Title
Cowboy Bebop                            0
Cowboy Bebop Tengoku no Tobira          1
Trigun                                  2
Witch Hunter Robin                      3
Bouken Ou Beet                          4
                                    ...  
Animagear                           12974
Magical Halloween MiracleQuartet    12975
Orbital Era                         12976
Akai Hana Shiroi Hana               12977
Arui Tekoteko                       12978
Length: 12979, dtype: int64

In [11]:
def get_recommendations(title, cosine_sim=cosine_sim):
    
    idx = indices[title]

    sim_scores = list(enumerate(cosine_sim[idx]))

    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 15 most similar movies
    sim_scores = sim_scores[1:16]
    
    anime_indices = [i[0] for i in sim_scores]

    return df['Title'].iloc[anime_indices]

In [12]:
get_recommendations('Sen to Chihiro no Kamikakushi')

9604                                      Shounen Maid
560                                 Kujibiki Unbalance
4521                               Suki Desu Suzukikun
5520             Kami nomi zo Shiru Sekai 4nin to Idol
3283                       KerakuNoOH King of Pleasure
10156                              Kushimitama Samurai
5869                                      Sankarea OVA
2422                             ef A Tale of Memories
2816                                         Kiss yori
4199              ef A Tale of Memories  Recollections
11903                                   Mirai no Mirai
740                                         Jyu Oh Sei
2669                                    Hatenkou Yuugi
4822     Ultraman Kids Haha wo Tazunete 3000man Kounen
5113                                Koguma no Korochan
Name: Title, dtype: object

In [13]:
get_recommendations('Koe no Katachi')

12407    Seishun Buta Yarou wa Yumemiru Shoujo no Yume ...
1254                                      Asatte no Houkou
5605                                          Hero Herokun
9159                                     Yuujou no Kickoff
1731                                                  Pops
6486                                          Danchi Tomoo
4730                        Fortune Arterial Akai Yakusoku
12702                               Tomo Tabidachi no Toki
6611                         Kakumeiki Valvrave 2nd Season
6473                                        Harisu no Kaze
8974                         Gakuen Handsome The Animation
10173                                      Gakuen Handsome
10787                                           Tejina Shi
5429                                    VitaminX Addiction
5830                             Yajikita Gakuen Douchuuki
Name: Title, dtype: object

Genres, Producer and Studio Based Recommender It goes without saying that the quality of our recommender would be increased with the usage of better metadata. That is exactly what we are going to do in this section. We are going to build a recommender based on the following metadata: the producer, related genres and the studio.

In [14]:
df.head(2)

,Anime_id,Title,Genre,Synopsis,Type,Producer,Studio,Rating,ScoredBy,Popularity,Members,Episodes,Source,Aired
0,1,Cowboy Bebop,"['Action', 'Adventure', 'Comedy', 'Drama', 'Sc...","In the year 2071, humanity has colonized sever...",TV,['Bandai Visual'],['Sunrise'],8.81,363889.0,39.0,704490.0,26.0,Original,"Apr 3, 1998 to Apr 24, 1999"
1,5,Cowboy Bebop Tengoku no Tobira,"['Action', 'Space', 'Drama', 'Mystery', 'Sci-Fi']","Another day, another bounty—such is the life o...",Movie,"['Sunrise', 'Bandai Visual']",['Bones'],8.41,111187.0,475.0,179899.0,1.0,Original,"Sep 1, 2001"


In [15]:
features = ['Genre','Producer', 'Studio']

print(df[features].isnull().sum())

Genre          0
Producer    5779
Studio      5611
dtype: int64


In [16]:
df[features] = df[features].fillna('[' ']')

In [17]:
from ast import literal_eval

for feature in features:
    df[feature] = df[feature].apply(literal_eval)

In [18]:
df.Type.unique()

array(['TV', 'Movie', 'OVA', 'Special', 'ONA', 'Music', 'Unknown'],
      dtype=object)

In [19]:
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ","")) for i in x]
    
    else:
        if isinstance(x, str):
            return str.lower(x.replace(" ",""))
        else:
            return ""

In [20]:
features = ['Genre','Producer', 'Studio', 'Type']

for feature in features:
    df[feature] = df[feature].apply(clean_data)

In [21]:
df.head(2)

,Anime_id,Title,Genre,Synopsis,Type,Producer,Studio,Rating,ScoredBy,Popularity,Members,Episodes,Source,Aired
0,1,Cowboy Bebop,"[action, adventure, comedy, drama, sci-fi, space]","In the year 2071, humanity has colonized sever...",tv,[bandaivisual],[sunrise],8.81,363889.0,39.0,704490.0,26.0,Original,"Apr 3, 1998 to Apr 24, 1999"
1,5,Cowboy Bebop Tengoku no Tobira,"[action, space, drama, mystery, sci-fi]","Another day, another bounty—such is the life o...",movie,"[sunrise, bandaivisual]",[bones],8.41,111187.0,475.0,179899.0,1.0,Original,"Sep 1, 2001"


In [22]:
features

['Genre', 'Producer', 'Studio', 'Type']

In [23]:
def create_soup(x):
    return " ".join(x['Genre']) + " " + x['Type'] + " " + " ".join(x['Producer']) + " " + " ".join(x['Studio']) + " " + x['Synopsis'] + " " + " ".join(x['Studio']) 

In [24]:
df['soup'] = df.apply(create_soup, axis=1)

In [25]:
df['soup']

0        action adventure comedy drama sci-fi space tv ...
1        action space drama mystery sci-fi movie sunris...
2        action sci-fi adventure comedy drama shounen t...
3        action magic police supernatural drama mystery...
4        adventure fantasy shounen supernatural tv  toe...
                               ...                        
12974    kids mecha ona   A promotional video for the n...
12975    comedy ecchi fantasy game music music   Bundle...
12976    action adventure fantasy sci-fi space movie   ...
12977    kids music music   Akai Hana Shiroi Hana  was ...
12978    kids music music   Of a pair of shoes, the lef...
Name: soup, Length: 12979, dtype: object

The next steps are the same as what we did with our plot description based recommender. One important difference is that we use the CountVectorizer() instead of TF-IDF. This is because we do not want to down-weight the presence of an producer if he or she has acted or directed in relatively more movies. It doesn't make much intuitive sense.

In [26]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(df['soup'])

from sklearn.metrics.pairwise import cosine_similarity

cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

In [27]:
df = df.reset_index()
indices = pd.Series(df.index, index=df['Title'])

In [28]:
indices

Title
Cowboy Bebop                            0
Cowboy Bebop Tengoku no Tobira          1
Trigun                                  2
Witch Hunter Robin                      3
Bouken Ou Beet                          4
                                    ...  
Animagear                           12974
Magical Halloween MiracleQuartet    12975
Orbital Era                         12976
Akai Hana Shiroi Hana               12977
Arui Tekoteko                       12978
Length: 12979, dtype: int64

In [29]:
get_recommendations('Cowboy Bebop', cosine_sim2)

1                           Cowboy Bebop Tengoku no Tobira
5488                                Planetes Picture Drama
2268          Ginga Tetsudou Monogatari Eien e no Bunkiten
4970                  Shin Mashin Eiyuuden Wataru Majinzan
3226                                  Ginga Senpuu Braiger
4089                   Sora wo Kakeru Shoujo Picture Drama
979                              Seihou Tenshi Angel Links
7542     Seihou Tenshi Angel Links Meifon no Special Ka...
6253       Mashin Eiyuuden Wataru Soukaizan Eiyuu Densetsu
10632            Mobile Suit Gundam Thunderbolt 2nd Season
6718                                  Dagram vs RoundFacer
1473               Dirty Pair no Ooshoubu Nolandia no Nazo
3836                                     Space Cobra Pilot
4428                    Wrestler Gundan Seisenshi Robin Jr
5017                         Zero Tester Chikyuu wo Mamore
Name: Title, dtype: object

In [30]:
get_recommendations('Sen to Chihiro no Kamikakushi', cosine_sim2)

740                                           Jyu Oh Sei
11903                                     Mirai no Mirai
9604                                        Shounen Maid
10156                                Kushimitama Samurai
4534                             Karigurashi no Arrietty
2669                                      Hatenkou Yuugi
7374     Kero Kero Keroppi no Yowamushiouji no Daibouken
9583                                     Kemushi no Boro
6438           Ashita Genki ni Nare Hanbun no Satsumaimo
5299                                Da Yu Hai Tang Movie
6115                      Majokko Shimai no Yoyo to Nene
999               Fushigi no Umi no Nadia Original Movie
6381                                                 Jib
5113                                  Koguma no Korochan
3124                                    Net Ghost Pipopa
Name: Title, dtype: object

In [31]:
get_recommendations('Mirai no Mirai', cosine_sim2)

3478                             Umineko no Naku Koro ni
3809                                   Suteneko Torachan
2269                                       Umi no Triton
4233                           Doubutsu Mura no Daisodou
9493                    Tenchi Muyou Ryououki 4th Season
11288                                        Yao Shen Ji
5123                                               Oshin
8323     Kyoukai no Kanata Movie 2 Ill Be Here  Miraihen
6529                                         Golden Time
6035                           Kono Sekai no Katasumi ni
298                            Mama wa Shougaku 4 Nensei
5766                        Ookami Kodomo no Ame to Yuki
8395                                           Hand Soap
6357                                 Dareka no Manazashi
1498                                      Hadashi no Gen
Name: Title, dtype: object

In [32]:
display(df[['Title', 'Rating', 'Producer', 'Studio']].loc[df['Title'] == 'Sen to Chihiro no Kamikakushi'])

,Title,Rating,Producer,Studio
178,Sen to Chihiro no Kamikakushi,8.92,[toho],[studioghibli]


### (2) Collaborative filtering using k-Nearest Neighbors (kNN)

In [33]:
rating = pd.read_csv('rating.csv', encoding='latin')

print('rating (shape):', rating.shape)

rating.head(5)

rating (shape): (7813737, 3)


,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1


In [35]:
print('df (shape):', df.shape)

df.head(2)

df (shape): (12979, 16)


,index,Anime_id,Title,Genre,Synopsis,Type,Producer,Studio,Rating,ScoredBy,Popularity,Members,Episodes,Source,Aired,soup
0,0,1,Cowboy Bebop,"[action, adventure, comedy, drama, sci-fi, space]","In the year 2071, humanity has colonized sever...",tv,[bandaivisual],[sunrise],8.81,363889.0,39.0,704490.0,26.0,Original,"Apr 3, 1998 to Apr 24, 1999",action adventure comedy drama sci-fi space tv ...
1,1,5,Cowboy Bebop Tengoku no Tobira,"[action, space, drama, mystery, sci-fi]","Another day, another bounty—such is the life o...",movie,"[sunrise, bandaivisual]",[bones],8.41,111187.0,475.0,179899.0,1.0,Original,"Sep 1, 2001",action space drama mystery sci-fi movie sunris...


In [36]:
rating.columns = ['User_id', 'Anime_id', 'Rating']

df.isnull().sum()

index         0
Anime_id      0
Title         1
Genre         0
Synopsis      0
Type          0
Producer      0
Studio        0
Rating        0
ScoredBy      0
Popularity    0
Members       0
Episodes      0
Source        0
Aired         0
soup          0
dtype: int64

Remove anime with low count of ratings and users who gave low count of ratings

In [37]:
anime_rating = rating.groupby(by = 'Anime_id').count()
anime_rating = anime_rating['Rating'].reset_index().rename(columns={'Rating':'Rating_count'})
anime_rating

,Anime_id,Rating_count
0,1,15509
1,5,6927
2,6,11077
3,7,2629
4,8,413
...,...,...
11195,34367,5
11196,34412,1
11197,34475,4
11198,34476,1


In [38]:
anime_rating['Rating_count'].describe()

count    11200.000000
mean       697.655089
std       2028.627749
min          1.000000
25%          5.000000
50%         51.500000
75%        385.250000
max      39340.000000
Name: Rating_count, dtype: float64

In [39]:
final_anime = anime_rating[anime_rating['Rating_count']>50]
final_anime.shape

(5625, 2)

In [40]:
user_rating = rating.groupby(by='User_id').count()
user_rating = user_rating['Rating'].reset_index().rename(columns={'Rating':'Rating_count'})
user_rating

,User_id,Rating_count
0,1,153
1,2,3
2,3,94
3,4,52
4,5,467
...,...,...
73510,73512,13
73511,73513,33
73512,73514,1
73513,73515,196


In [41]:
final_user = user_rating[user_rating['Rating_count']>80]
final_user.shape

(29368, 2)

Dataset with popular anime and high rating counts from users

In [42]:
final_anime_dt = rating[rating['Anime_id'].isin(final_anime['Anime_id'])]
final_dt = final_anime_dt[final_anime_dt['User_id'].isin(final_user['User_id'])]
final_dt.head()

,User_id,Anime_id,Rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1


### Construct Rating Matrix

We will construct a matrix wherein Anime id will be indexes and User id in columns and then Convert rating matrix to csr matrix to save memory

In [43]:
rating_matrix = final_dt.pivot_table(index='Anime_id',columns='User_id',values='Rating').fillna(0)

print(rating_matrix.shape)
rating_matrix.head()

(5625, 29368)


User_id,1,3,5,7,11,13,14,17,21,29,...,73494,73495,73499,73500,73502,73503,73504,73507,73510,73515
Anime_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,9.0,0.0,...,0.0,10.0,9.0,0.0,0.0,9.0,10.0,9.0,0.0,10.0
5,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,9.0,0.0,...,0.0,8.0,0.0,0.0,0.0,7.0,10.0,8.0,0.0,10.0
6,0.0,0.0,8.0,0.0,0.0,-1.0,0.0,7.0,0.0,0.0,...,9.0,-1.0,9.0,0.0,0.0,9.0,9.0,9.0,0.0,10.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,7.0,0.0,0.0,9.0,0.0,7.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [44]:
from scipy.sparse import csr_matrix
csr_rating_matrix =  csr_matrix(rating_matrix.values)

print(csr_rating_matrix)

  (0, 5)	-1.0
  (0, 8)	9.0
  (0, 11)	7.0
  (0, 16)	10.0
  (0, 18)	10.0
  (0, 20)	10.0
  (0, 21)	10.0
  (0, 22)	-1.0
  (0, 29)	10.0
  (0, 30)	8.0
  (0, 46)	9.0
  (0, 49)	10.0
  (0, 53)	10.0
  (0, 56)	9.0
  (0, 57)	8.0
  (0, 60)	7.0
  (0, 63)	7.0
  (0, 65)	8.0
  (0, 69)	10.0
  (0, 71)	9.0
  (0, 72)	8.0
  (0, 73)	10.0
  (0, 76)	9.0
  (0, 77)	8.0
  (0, 78)	9.0
  :	:
  (5624, 28977)	-1.0
  (5624, 28992)	10.0
  (5624, 29022)	-1.0
  (5624, 29027)	10.0
  (5624, 29035)	8.0
  (5624, 29037)	-1.0
  (5624, 29049)	8.0
  (5624, 29073)	-1.0
  (5624, 29076)	8.0
  (5624, 29079)	8.0
  (5624, 29094)	-1.0
  (5624, 29104)	9.0
  (5624, 29114)	8.0
  (5624, 29128)	7.0
  (5624, 29129)	8.0
  (5624, 29131)	-1.0
  (5624, 29140)	9.0
  (5624, 29157)	-1.0
  (5624, 29215)	8.0
  (5624, 29221)	-1.0
  (5624, 29234)	8.0
  (5624, 29258)	10.0
  (5624, 29264)	8.0
  (5624, 29292)	9.0
  (5624, 29317)	9.0


#### Fit the matrix into k-Nearest Neighbors (kNN)

We decided to use cosine similarity as the metric for the algorithm

In [45]:
from sklearn.neighbors import NearestNeighbors

recommender = NearestNeighbors(metric='cosine')

recommender.fit(csr_rating_matrix)

NearestNeighbors(metric='cosine')

In [46]:
user_anime = df[df['Title']=='Sen to Chihiro no Kamikakushi']
user_anime

,index,Anime_id,Title,Genre,Synopsis,Type,Producer,Studio,Rating,ScoredBy,Popularity,Members,Episodes,Source,Aired,soup
178,178,199,Sen to Chihiro no Kamikakushi,"[adventure, supernatural, drama]","Stubborn, spoiled, and naïve, 10-year-old Chih...",movie,[toho],[studioghibli],8.92,496902.0,40.0,703229.0,1.0,Original,"Jul 20, 2001",adventure supernatural drama movie toho studio...


In [47]:
import numpy as np

In [48]:
user_anime_index = np.where(rating_matrix.index==int(user_anime['Anime_id']))[0][0]

# this index is from rating matrix not from the anime dataset!

print('rating natrix index:', user_anime_index)

# getting the ratings based on the index

user_anime_ratings = rating_matrix.iloc[user_anime_index]
user_anime_ratings

rating natrix index: 173


User_id
1         0.0
3        10.0
5         8.0
7         0.0
11       10.0
         ... 
73503     8.0
73504     8.0
73507    10.0
73510    -1.0
73515     0.0
Name: 199, Length: 29368, dtype: float64

In [50]:
#now we need to convert this into 2d array (with only 1 row)

user_anime_ratings_reshaped = user_anime_ratings.values.reshape(1,-1)
user_anime_ratings_reshaped

array([[ 0., 10.,  8., ..., 10., -1.,  0.]])

In [51]:
# the ratings will be plotted and will return 11 indices and distances of nearest neighbors
distances, indices = recommender.kneighbors(user_anime_ratings_reshaped,n_neighbors=16)

print(distances)

[[0.         0.26155118 0.30075682 0.35429596 0.41723165 0.41745461
  0.45026117 0.45202544 0.47006025 0.47340618 0.47559998 0.47617122
  0.48209854 0.48600216 0.48878719 0.49086589]]


In [52]:
print(indices)

[[ 173  394  140  480 1316 1799  469  470 1431   98 1349  527  199  740
    18 4444]]


In [53]:
# the returned indices will be used to get anime id(index) on rating matrix, 
# excluding the first element since the first nearest neighbor is itself

nearest_neighbors_indices = rating_matrix.iloc[indices[0]].index[1:]

In [55]:
nearest_neighbors = pd.DataFrame({'Anime_id': nearest_neighbors_indices})
pd.merge(nearest_neighbors,df,on='Anime_id',how='left')

,Anime_id,index,Title,Genre,Synopsis,Type,Producer,Studio,Rating,ScoredBy,Popularity,Members,Episodes,Source,Aired,soup
0,431,372.0,Howl no Ugoku Shiro,"[adventure, drama, fantasy, romance]","That jumbled piece of architecture, that cacop...",movie,"[d-rights, tokumashoten]",[studioghibli],8.73,333104.0,88.0,479791.0,1.0,Novel,"Nov 20, 2004",adventure drama fantasy romance movie d-rights...
1,164,144.0,Mononoke Hime,"[action, adventure, fantasy]",When an Emishi village is attacked by a fierce...,movie,"[tokumashoten, nippontelevisionnetwork]",[studioghibli],8.80,332946.0,77.0,506368.0,1.0,Original,"Jul 12, 1997",action adventure fantasy movie tokumashoten ni...
2,523,432.0,Tonari no Totoro,"[adventure, comedy, supernatural]","In 1950s Japan, Tatsuo Kusakabe relocates hims...",movie,[studiofantasia],[studioghibli],8.45,261626.0,120.0,403817.0,1.0,Original,"Apr 16, 1988",adventure comedy supernatural movie studiofant...
3,1535,1239.0,Death Note,"[mystery, police, psychological, supernatural,...","A shinigami, as a god of death, can kill any p...",tv,"[vap, konami, ashiproduction, nippontelevision...",[madhouse],8.67,1006242.0,1.0,1451708.0,37.0,Manga,"Oct 4, 2006 to Jun 27, 2007",mystery police psychological supernatural thri...
4,2236,1865.0,Toki wo Kakeru Shoujo,"[sci-fi, adventure, drama, romance]",Makoto Konno is in her last year of high schoo...,movie,"[kadokawashoten, bandai, memory-tech, happinet...",[madhouse],8.35,248870.0,130.0,410159.0,1.0,Novel,"Jul 15, 2006",sci-fi adventure drama romance movie kadokawas...
5,512,425.0,Majo no Takkyuubin,"[adventure, comedy, drama, magic, romance, fan...","Kiki, a 13-year-old witch-in-training, must sp...",movie,[],"[studioghibli, studiohibari]",8.29,139453.0,361.0,214986.0,1.0,Novel,"Jul 29, 1989",adventure comedy drama magic romance fantasy m...
6,513,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1689,1374.0,Byousoku 5 Centimeter,"[drama, romance, sliceoflife]","Takaki Toono and Akari Shinohara, two very clo...",movie,[],[comixwavefilms],7.95,280263.0,108.0,456478.0,3.0,Original,"Mar 3, 2007",drama romance sliceoflife movie comixwavefilm...
8,121,101.0,Fullmetal Alchemist,"[action, adventure, comedy, drama, fantasy, ma...","Edward Elric, a young, brilliant alchemist, ha...",tv,"[aniplex, dentsu, squareenix, mainichibroadcas...",[bones],8.29,490243.0,37.0,789593.0,51.0,Manga,"Oct 4, 2003 to Oct 2, 2004",action adventure comedy drama fantasy magic mi...
9,1575,1274.0,Code Geass Hangyaku no Lelouch,"[action, military, sci-fi, superpower, drama, ...","In the year 2010, the Holy Empire of Britannia...",tv,"[bandaivisual, mainichibroadcastingsystem, ban...",[sunrise],8.79,625466.0,11.0,983674.0,25.0,Original,"Oct 6, 2006 to Jul 29, 2007",action military sci-fi superpower drama mecha ...
